In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes #
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [13]:
from bs4 import BeautifulSoup
import requests
import csv

In [14]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(website_url.text, "html.parser")

In [15]:
my_table = soup.find("table",{"class":"wikitable sortable"}).tbody

In [16]:
rows = my_table.find_all("tr")

columns = [a.text.replace("\n"," ") for a in rows[0].find_all("th")]

df = pd.DataFrame(columns=columns)

for i in range(1, len(rows)):
    tds = rows[i].find_all("td")
    
    if len(tds) == 3:
        values= [tds[0].text, tds[1].text, tds[2].text.replace("\n", "".replace("\xa0",""))]
        
        
    else:
        values = [td.text.replace("\n", '').replace("\xa0", '') for td in tds]

    df = df.append(pd.Series(values, index=columns), ignore_index=True)

In [17]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
df.columns = ['Postcode', 'Borough', "Neighborhood"]

In [19]:
df["Borough"].replace('Not assigned', np.nan, inplace=True)
df.dropna(inplace=True)
df= df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df.loc[df['Neighborhood']=="Not assigned",'Neighborhood']=df.loc[df['Neighborhood']=="Not assigned",'Borough']
df.reset_index(inplace=True)
df.shape   

(103, 3)

In [20]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('toronto_coordinates.csv')
coors.columns= ["Postcode","Latitude","Longitude"]
print('Coordinates downloaded!')
coors

Coordinates downloaded!
Coordinates downloaded!


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [21]:
toronto_final_df  = df.merge(coors, on="Postcode", how = 'inner')

toronto_final_df


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [265]:
toronto_df = toronto_final_df


In [26]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [27]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#4386cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [28]:
##Define Foursquare Credentials and Version
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = 'xxx' # Foursquare API version
LIMIT = 100    

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxx
CLIENT_SECRET:xxx


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
toronto_venues = getNearbyVenues( names = toronto_df['Neighborhood'],
                                  latitudes = toronto_df['Latitude'],
                                  longitudes = toronto_df['Longitude']
                                  )



Malvern,Rouge
Port Union,Rouge Hill,Highland Creek
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
Ionview,Kennedy Park,East Birchmount Park
Oakridge,Clairlea,Golden Mile
Scarborough Village West,Cliffcrest,Cliffside
Birch Cliff,Cliffside West
Scarborough Town Centre,Dorset Park,Wexford Heights
Maryvale,Wexford
Agincourt
Tam O'Shanter,Sullivan,Clarks Corners
Milliken,L'Amoreaux East,Steeles East,Agincourt North
L'Amoreaux West
Upper Rouge
Hillcrest Village
Henry Farm,Fairview,Oriole
Bayview Village
York Mills,Silver Hills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Downsview North,Bathurst Manor,Wilson Heights
York University,Northwood Park
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [266]:
toronto_venues_final  = toronto_venues.merge(toronto_df[["Borough","Neighborhood"]], on="Neighborhood", how = 'inner')
toronto_venues_final = toronto_venues_final[(toronto_venues_final['Borough'].str.contains('Toronto')) | (toronto_venues_final['Borough'].str.contains('East'))]

toronto_venues_final['Borough'].unique()


array(['East York', 'East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [267]:
toronto_venues_final.drop("Borough",axis=1,inplace=True)
toronto_venues_final.head()
toronto_df = toronto_df[(toronto_df['Borough'].str.contains('Toronto')) | (toronto_df['Borough'].str.contains('East'))]

toronto_df.drop("Borough",axis=1,inplace=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [268]:
###3. Analyze Each Neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_final[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_final['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

Toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()


In [269]:
Toronto_grouped.set_index("Neighborhood",inplace=True)
Toronto_grouped.reset_index(inplace=True)
Toronto_grouped.head()



,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Bathurst Quay,Island airport,King and Spadina,...",0.000000,0.0,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.062500,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0625,0.0,0.0625,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.062500,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00

In [270]:
num_top_venues = 10

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    top5=temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)
    top5.index = np.arange(1,len(top5)+1)
    print(top5)
    print('\n')


----Bathurst Quay,Island airport,King and Spadina,Railway Lands,CN Tower,Harbourfront West,South Niagara----
               venue  freq
1   Airport Terminal  0.12
2     Airport Lounge  0.12
3    Airport Service  0.12
4              Plane  0.06
5    Harbor / Marina  0.06
6        Coffee Shop  0.06
7      Boat or Ferry  0.06
8           Boutique  0.06
9   Sculpture Garden  0.06
10      Airport Gate  0.06


----Berczy Park----
                 venue  freq
1          Coffee Shop  0.09
2         Cocktail Bar  0.05
3   Seafood Restaurant  0.04
4           Steakhouse  0.04
5                 Café  0.04
6               Bakery  0.04
7          Cheese Shop  0.04
8   Italian Restaurant  0.04
9             Beer Bar  0.04
10      Farmers Market  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                   venue  freq
1            Yoga Studio  0.05
2          Auto Workshop  0.05
3             Comic Shop  0.05
4                   Park  0.05
5       Recording Studio  0.05
6      

In [271]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [352]:
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,"Bathurst Quay,Island airport,King and Spadina,...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Coffee Shop,Sculpture Garden,Boat or Ferry,Plane,Airport Gate,Airport Food Court,Airport,Boutique,Bar,Doner Restaurant,Farmers Market,Falafel Restaurant,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dessert Shop,Dog Run,Discount Store,Diner,Fish & Chips Shop,Department Store,Deli / Bodega
1,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Cheese Shop,Café,Farmers Market,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Comfort Food Restaurant,Restaurant,Shopping Mall,Clothing Store,Bistro,Park,Breakfast Spot,Nightclub,Concert Hall,Museum,Creperie,Diner,Eastern European Restaurant,Liquor Store,Jazz Club,Fish Market,Irish Pub,Fountain,Hotel,French Restaurant
2,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Recording Studio,Smoke Shop,Skate Park,Brewery,Burrito Place,Butcher,Restaurant,Park,Auto Workshop,Comic Shop,Farmers Market,Fast Food Restaurant,Light Rail Station,Garden,Garden Center,Spa,Pizza Place,Gym / Fitness Center,Dog Run,Airport,Airport Food Court,Airport Gate,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,Burger Joint,Japanese Restaurant,Chinese Restaurant,Bakery,Bar,Spa,Sushi Restaurant,Indian Restaurant,Ice Cream Shop,Bubble Tea Shop,Restaurant,Salad Place,Clothing Store,Portuguese Restaurant,Pizza Place,Ramen Restaurant,Gastropub,Office,Poke Place,Yoga Studio,Comic Shop,Seafood Restaurant,Miscellaneous Shop,Department Store,Dessert Shop
4,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Convenience Store,Coffee Shop,Diner,Italian Restaurant,Restaurant,Discount Store,Dog Run,Women's Store,Doner Restaurant,Donut Shop,Dessert Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Dumpling Restaurant,Deli / Bodega,Department Store,Farmers Market,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space


In [353]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 0, 6, 0, 0, 0, 0, 6, 0, 6], dtype=int32)

In [354]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
35,M4B,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0,Pizza Place,Fast Food Restaurant,Gastropub,Pet Store,Café,Breakfast Spot,Pharmacy,Bank,Athletics & Sports,Intersection,Gym / Fitness Center,Gay Bar,Greek Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Grocery Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space
36,M4C,Woodbine Heights,43.695344,-79.318389,6,Skating Rink,Cosmetics Shop,Curling Ice,Bus Stop,Athletics & Sports,Park,Beer Store,Pharmacy,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Women's Store,Diner,Discount Store,Fast Food Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop
37,M4E,The Beaches,43.676357,-79.293031,6,Health Food Store,Other Great Outdoors,Trail,Pub,Ethiopian Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Department Store,Falafel Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Dance Studio,Deli / Bodega,Fast Food Restaurant,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store,Concert Hall,Comic Shop
38,M4G,Leaside,43.709060,-79.363452,6,Coffee Shop,Sporting Goods Shop,Grocery Store,Burger Joint,Sushi Restaurant,Furniture / Home Store,Liquor Store,Bank,Sports Bar,Fish & Chips Shop,Beer Store,Bike Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Brewery,Electronics Store,Restaurant,Mexican Restaurant,Dessert Shop,Clothing Store,Pet Store,Supermarket,Smoothie Shop,Food & Drink Shop,Gym,Bagel Shop,Costume Shop,Coworking Space,Eastern European Restaurant
39,M4H,Thorncliffe Park,43.705369,-79.349372,6,Indian Restaurant,Yoga Studio,Supermarket,Housing Development,Intersection,Liquor Store,Discount Store,Park,Pharmacy,Grocery Store,Coffee Shop,Sandwich Place,Burger Joint,Bank,Pizza Place,Gym,Warehouse Store,Fish & Chips Shop,Fast Food Restaurant,Airport,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [355]:
len(toronto_merged)

43

In [356]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [357]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
35,"Woodbine Gardens,Parkview Hill",Pizza Place,Fast Food Restaurant,Gastropub,Pet Store,Café,Breakfast Spot,Pharmacy,Bank,Athletics & Sports,Intersection,Gym / Fitness Center,Gay Bar,Greek Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Grocery Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space
43,Studio District,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant,Sandwich Place,Cheese Shop,Chinese Restaurant,Clothing Store,Convenience Store,Park,Stationery Store,Coworking Space,Music Store,Middle Eastern Restaurant,Latin American Restaurant,Fish Market,Ice Cream Shop,Gym / Fitness Center,Bookstore,Gym,Bank,Bar,Basketball Stadium,Flea Market
47,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Restaurant,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Thai Restaurant,Park,Brewery,Greek Restaurant,Pharmacy,Deli / Bodega,Costume Shop,Diner,Discount Store,Farmers Market,Flower Shop,Indian Restaurant,Fried Chicken Joint,Gourmet Shop,Seafood Restaurant,Gym,Toy / Game Store,Bagel Shop,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Airport Lounge
51,"St. James Town,Cabbagetown",Coffee Shop,Restaurant,Italian Restaurant,Café,Pub,Park,Bakery,Pizza Place,Jewelry Store,Bar,Chinese Restaurant,Japanese Restaurant,Diner,Beer Store,Pet Store,Caribbean Restaurant,Snack Place,Farmers Market,Breakfast Spot,Butcher,Liquor Store,Sandwich Place,Bank,Market,Flower Shop,Deli / Bodega,Taiwanese Restaurant,Playground,Grocery Store,Pharmacy
52,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Bubble Tea Shop,Mediterranean Restaurant,Café,Burger Joint,Men's Store,Fast Food Restaurant,Pub,Gym,Gastropub,Breakfast Spot,Mexican Restaurant,Burrito Place,Pizza Place,Pet Store,Persian Restaurant,Park,Nightclub,Ramen Restaurant,Bookstore,Yoga Studio,Dance Studio,Creperie,Salon / Barbershop,Juice Bar
53,"Harbourfront,Regent Park",Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Restaurant,Breakfast Spot,Café,Theater,Hotel,Gym / Fitness Center,Health Food Store,Historic Site,Italian Restaurant,Farmers Market,Performing Arts Venue,French Restaurant,Dessert Shop,Event Space,Electronics Store,Spa,Cosmetics Shop,Chocolate Shop,Brewery,Beer Store,Bank,Art Gallery,Antique Shop,Shoe Store,Yoga Studio
54,"Ryerson,Garden District",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Diner,Tea Room,Theater,Pizza Place,Ramen Restaurant,Restaurant,Pub,Hotel,Bookstore,Hookah Bar,Lounge,Dessert Shop,Plaza,Department Store,Sandwich Place,Poutine Place,Other Great Outdoors,Burger Joint,Burrito Place,Shoe Store,Chinese Restaurant
55,St. James Town,Café,Coffee Shop,Restaurant,Hotel,Breakfast Spot,Gastropub,Clothing Store,Cocktail Bar,Cosmetics Shop,Bakery,Italian Restaurant,Seafood Restaurant,Pizza Place,Beer Bar,Japanese Restaurant,Park,Diner,Farmers Market,American Restaurant,Gym,Fish Market,German Restaurant,Furniture / Home Store,French Restaurant,Poke Place,Camera Store,Cheese Shop,Grocery Store,Church,Performing 

## Cluster 2

In [358]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
63,Roselawn,Ice Cream Shop,Garden,Women's Store,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Department Store,Filipino Restaurant,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store,Concert Hall,Comic Shop


## Cluster 3

In [359]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
48,"Moore Park,Summerhill East",Playground,Tennis Court,Comfort Food Restaurant,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Department Store,Comic Shop,Filipino Restaurant,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store,Concert Hall


## Cluster 4

In [360]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
50,Rosedale,Park,Playground,Trail,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Dance Studio,Deli / Bodega,Fast Food Restaurant,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store,Concert Hall,Comic Shop,Comfort Food Restaurant


## Cluster 5

In [361]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
44,Lawrence Park,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Department Store,Fast Food Restaurant,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store,Concert Hall,Comic Shop


## Cluster 6

In [362]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
40,East Toronto,Convenience Store,Coffee Shop,Park,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Department Store,Dessert Shop,Filipino Restaurant,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Concert Hall,Fast Food Restaurant


## Cluster 7

In [363]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
36,Woodbine Heights,Skating Rink,Cosmetics Shop,Curling Ice,Bus Stop,Athletics & Sports,Park,Beer Store,Pharmacy,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Women's Store,Diner,Discount Store,Fast Food Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop
37,The Beaches,Health Food Store,Other Great Outdoors,Trail,Pub,Ethiopian Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Department Store,Falafel Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Dance Studio,Deli / Bodega,Fast Food Restaurant,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store,Concert Hall,Comic Shop
38,Leaside,Coffee Shop,Sporting Goods Shop,Grocery Store,Burger Joint,Sushi Restaurant,Furniture / Home Store,Liquor Store,Bank,Sports Bar,Fish & Chips Shop,Beer Store,Bike Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Brewery,Electronics Store,Restaurant,Mexican Restaurant,Dessert Shop,Clothing Store,Pet Store,Supermarket,Smoothie Shop,Food & Drink Shop,Gym,Bagel Shop,Costume Shop,Coworking Space,Eastern European Restaurant
39,Thorncliffe Park,Indian Restaurant,Yoga Studio,Supermarket,Housing Development,Intersection,Liquor Store,Discount Store,Park,Pharmacy,Grocery Store,Coffee Shop,Sandwich Place,Burger Joint,Bank,Pizza Place,Gym,Warehouse Store,Fish & Chips Shop,Fast Food Restaurant,Airport,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
41,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Brewery,Bubble Tea Shop,Café,Restaurant,Caribbean Restaurant,Pub,Cosmetics Shop,Grocery Store,Bakery,Dessert Shop,Diner,Liquor Store,Juice Bar,Indian Restaurant,Fruit & Vegetable Store,Spa,Yoga Studio,Pizza Place,American Restaurant,Trail,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant
42,"The Beaches West,India Bazaar",Park,Pizza Place,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Fish & Chips Shop,Fast Food Restaurant,Liquor Store,Movie Theater,Pet Store,Pub,Sandwich Place,Burrito Place,Burger Joint,Brewery,Steakhouse,Gym,Athletics & Sports,Discount Store,Farmers Market,Falafel Restaurant,Airport Gate,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
45,Davisville North,Playground,Gym,Grocery Store,Sandwich Place,Breakfast Spot,Park,Hotel,Clothing Store,Food & Drink Shop,Dog Run,Diner,Discount Store,Dumpling Restaurant,Doner Restaurant,Donut Shop,Department Store,Eastern European Restaurant,Electronics Store,Dessert Shop,Curling Ice,Deli / Bodega,Dance Studio,Event Space,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store
46,North Toronto West,Coffee Shop,Yoga Studio,Bagel Shop,Gym / Fitness Center,Clothing Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Salon / Barbershop,Diner,Mexican Restaurant,

## Cluster 8

In [364]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
64,"Forest Hill North,Forest Hill West",Trail,Jewelry Store,Park,Sushi Restaurant,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Diner,Department Store,Fast Food Restaurant,Deli / Bodega,Dance Studio,Curling Ice,Cupcake Shop,Cuban Restaurant,Creperie,Coworking Space,Costume Shop,Cosmetics Shop,Convenience Store,Concert Hall
